In [1]:
from rikai.spark.utils import init_spark_session

rikai_version='0.0.20'

spark = init_spark_session(
    dict(
        [
            (
                "spark.jars.packages",
                ",".join(
                    [
                        "ai.eto:rikai_2.12:{}".format(rikai_version),
                    ]
                ),
            ),
            (
                "spark.rikai.sql.ml.registry.torchhub.impl",
                "ai.eto.rikai.sql.model.torchhub.TorchHubRegistry",
            ),
        ]
    )
)

:: loading settings :: url = jar:file:/Users/da/.pyenv/versions/3.8.10/envs/rikai-example/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/da/.ivy2/cache
The jars for the packages stored in: /Users/da/.ivy2/jars
ai.eto#rikai_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-aacc0286-d7c1-49df-acea-d616937c43e3;1.0
	confs: [default]
	found ai.eto#rikai_2.12;0.0.20 in central
	found org.antlr#antlr4-runtime;4.8-1 in local-m2-cache
	found com.thoughtworks.enableIf#enableif_2.12;1.1.7 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found com.typesafe.scala-logging#scala-logging_2.12;3.9.4 in central
	found org.slf4j#slf4j-api;1.7.30 in spark-list
	found io.circe#circe-core_2.12;0.12.3 in central
	found io.circe#circe-numbers_2.12;0.12.3 in central
	found org.typelevel#cats-core_2.12;2.0.0 in central
	found org.typelevel#cats-macros_2.12;2.0.0 in central
	found org.typelevel#cats-kernel_2.12;2.0.0 in central
	found io.circe#circe-generic_2.12;0.12.3 in central
	found com.chuusai#shapeless_2.12;2.3.3 in spark-list
	found org.typeleve

In [5]:
from rikai.spark.functions import to_image
spark.udf.register("to_image", to_image)

import torchvision
torchvision_v = f"v{torchvision.__version__.split('+', maxsplit=1)[0]}"
torchvision_v

22/01/14 14:27:52 WARN SimpleFunctionRegistry: The function to_image replaced a previously registered function.


'v0.9.1'

In [6]:
spark.sql("""
CREATE MODEL resnet18
USING "torchhub:///pytorch/vision:v0.9.1/resnet18";
""")

DataFrame[]

In [7]:
result = spark.sql("""
select ML_PREDICT(resnet18, to_image('https://github.com/eto-ai/rikai/raw/main/python/tests/assets/test_image.jpg')) as pred
"""
)

In [8]:
result.show()

Using cache found in /Users/da/.cache/torch/hub/pytorch_vision_v0.9.10 + 1) / 1]


+--------------------+
|                pred|
+--------------------+
|[0.8495411, -1.58...|
+--------------------+

